# RAG 평가 개요
- RAG 평가란 RAG 시스템이 주어진 입력에 대해 얼마나 효과적으로 관련 정보를 검색하고, 이를 기반으로 정확하고 유의미한 응답을 생성하는지를 측정하는 과정이다. 
- **평가 요소**
    - **검색 단계 평가**
        - 입력 질문에 대해 검색된 문서나 정보의 관련성과 정확성을 평가.
    - **생성 단계 평가**
        - 검색된 정보를 기반으로 생성된 응답의 품질, 정확성등을 평가.
- **평가 방법**
    - 온/오프라인 평가
        1. **오프라인 평가**
            - 미리 준비된 데이터셋을 활용하여 RAG 시스템의 성능을 측정한다.
        2. **온라인 평가**
            - 실제 사용자 트래픽과 피드백을 기반으로 시스템의 실시간 성능을 평가한다.
    - 정량적/정성적 평가
        1. 정량적 평가
            - 자동화된 지표를 사용하여 생성된 텍스트의 품질을 평가한다.
        2. 정성적 평가
            - 전문가나 일반 사용자가 직접 생성된 응답의 품질을 평가하여 주관적인 지표를 평가한다.

# [RAGAS](https://www.ragas.io/)
- RAGAS는 RAG 파이프라인을 **정량적 으로 평가하는** 오픈소스 프레임 워크이다. 
- RAGAS 문서: https://docs.ragas.io/en/stable/
## 설치
- `pip install ragas rapidfuzz`

In [9]:
!uv pip install ragas rapidfuzz

Resolved 102 packages in 486ms
Prepared 14 packages in 621ms
Uninstalled 2 packages in 52ms
Installed 15 packages in 577ms
 + appdirs==1.4.4
 + cfgv==3.5.0
 + datasets==4.4.2
 + diskcache==5.6.3
 + distlib==0.4.0
 - fsspec==2025.12.0
 + fsspec==2025.10.0
 + identify==2.6.15
 + instructor==1.13.0
 - jiter==0.12.0
 + jiter==0.11.1
 + nodeenv==1.10.0
 + pre-commit==4.5.1
 + ragas==0.4.2
 + scikit-network==0.33.5
 + ty==0.0.8
 + virtualenv==20.35.4


## RAGAS 평가 지표 개요
![ragas_score](figures/ragas_score.png)
- **Generation**
    - llm 모델이 생성한 답변에 대한 평가 지표들.
    - **Faithfulness(신뢰성)**
        -  생성된 답변과 검색된 문서(context)간의 관련성을 평가하는 지표
        -  생성된 답변이 주어진 문맥(context)에 얼마나 충실한지를 평가하는 지표로 할루시네이션에 대한 평가로 볼 수있다.
    - **Answer relevancy(답변 적합성)**
        - 생성된 답변과 사용자의 질문간의 관련성을 평가하는 지표
        - 생성된 답변이 사용자의 질문과 얼마나 관련성이 있는지를 평가하는 지표.
- **Retrieval**
    -  질문에 대해 검색한 문서(context)들에 대한 평가
    -  **Context Precision(문맥 정밀도)**
        -  검색된 문서(context)들 중 질문과 관련 있는 것들이 **얼마나 상위 순위에 위치하는지** 평가하는 지표.
    -  **Context Recall(문맥 재현률)**
        -  검색된 문서(context)가 정답(ground-truth)의 정보를 얼마나 포함하고 있는지 평가하는 지표.
- 이러한 지표들은 RAG 파이프라인의 성능을 다각도로 평가하는 데 활용된다.
![RAGAS_score2](figures/RAGAS_score2.png)

## 주요 평가지표
### Generation 평가
- LLM이 생성한 답변에 대한 평가
  
#### Faithfulness (신뢰성)
- 생성된 답변이 얼마나 주어진 검색 문서들(context)를 잘 반영해서 생성되었는지 평가한다. 할루시네이션에 대한 평가라고 할 수 있다. 
- 점수범위: **0 ~ 1** (1에 가까울수록 좋음)
- 답변에 포함된 모든 주장이 context에서 얼마나 추출 가능한지를 확인한다.

##### 평가 방법
1. Answer에서 주장 구문(claim statement)들을 생성(추출)한다. (주장이란, 질문(user input)과 관련된 내용)
    - 예) 
        - **질문**: 한국의 수도는 어디이고 인구는 얼마나 되나요? 
        - **LLM 답변**: 한국의 수도는 서울이고 인구수는 3000만명이다. 
        - **주장(claim)**: 
            1. 한국의 수도는 서울이다.
            2. 인구수는 3000만명이다.
2. 각 주장들을 context로 부터 추론 가능한지 판단한다. 이를 바탕으로 faithfulness 점수를 계산한다.
    - 예)
        - context: 한국은 동아시아에 위치하고 있는 나라다. 한국의 수도는 서울이다. .... 한국의 인구는 5000만명이고 서울에 1000만이 살고 있다.
        - 위 context에서 추론 가능한 주장: 
            - 한국의 수도는 서울이다. -> context에서 추론가능한 주장.
            - 한국의 인구는 3000만명이다. -> context에서 추론 불가능한 주장.
3. **Faithfulness score** 를 계산한다. 총 주장 수 중에서 context로 부터 추론가능한 주장의 개수.    
    - 예)
        - Faithfulness Score = $\cfrac{1}{2} = 0.5$ (두 개의 주장 중 한 개의 주장만 context에서 유추할 수있다.)
    - LLM 답변에서 주장을 추출 하는 것과 각 주장이 context에서 추론 가능한 지를 판단하는 것은 LLM 을 활용한다.
- 공식
    $$
    \text{Faithfulness Score}\;=\;\cfrac{\text{주어진\;context\;에서\;추론할\;수\;있는\;주장의\;개수}}{\text{총\;주장\;개수}}
    $$

### Answer relevancy (답변 적합성)
- 생성된 답변이 질문(user input)에 얼마나 잘 부합하는 지를 평가한다.
- 점수 범위: -1~1 (1에 가까울수록 좋음)
- LLM이 생성한 답변을 기반으로 질문들을 생성한다. 이렇게 생성한 질문들과 실제 질문(user input) 간의 유사도를 측정한다.

#### 평가방법
1. LLM이 생성한 답변을 기반으로 질문들을 생성한다.
    - 예) 
        - **LLM** 답변: 한국의 수도는 서울이고 인구수는 3000만명이다. 
        - **생성된 질문**: 
            1. 한국의 수도는 어디이고 인구는 얼마나 되나요?
            2. 한국의 수도는 어디인가요?
            3. 한국의 인구는 얼마나 되나요?
2. 실제 질문과 생성한 질문간의 코사인 유사도를 측정한다. 그 평균이 최종 점수가 된다.
    - 예)
        - **실제 질문**: 한국의 수도는 어디이고 인구는 얼마나 되나요?
        - **생성된 질문**: 
            1. 한국의 수도는 어디이고 인구는 얼마나 되나요?
            2. 한국의 수도는 어디인가요?
            3. 한국의 인구는 얼마나 되나요?
- 공식
  $$
    \cfrac{1}{N} \sum_{i=1}^{N} \text{cosine\_similarity}(q_{\text{user}_{_i}}, q_{\text{generated}})
  $$

## Retrieval 평가
Vector store에서 검색한 context에 대한 평가

### Context Precision
- 검색된 문서(context)들 중 질문과 관련 있는 것들이 얼마나 **상위 순위**에 있는 지 평가.
- 점수 범위: 0~1 (1에 가까울수록 좋음)


#### 평가방법

- 공식
$$
 \text{Context\;Precision@K} = \frac{\sum_{k=1}^{K} \left( \text{Precision@k} \times v_k \right)}{\ 상위\;K개\;결과에서의\;관련\;항목\;수}
$$
$$
 \text{Precision@k} = \frac{\text{True\;positive@k}}{(\text{True\;positive@k} + \text{False\;positive@k})} \\
$$
- $\text{Precision@k}$: 개별 문서에 대한 Precision
- K: context 의 개수(chuck 수)
- $v_k$: 관련성 여부로 0 또는 1. (0: 관련 없음, 1: 관련 있음)

#### 예시
- 질문과 context 관련성 예
    - 질문: 한국의 수도는 어디이고 인구는 얼마나 되나요?
    - 높은 정밀도 context
        - 한국의 수도는 서울이고 인구는 5000명 입니다. 
        - 한국의 수도는 서울입니다.
        - 한국은 동아시아에 위치해 있는 국가로 수도는 서울입니다.
        - 한국의 인구는 5000만명 입니다.
    - 낮은 정밀도 context
        - 한국은 동아시아에 위치한 국가입니다.
        - 한국의 K-pop은 전 세계적으로 유명합니다.
        - 비빔밥, 불고기는 한국의 대표적인 음식입니다.
    - **높은 정밀도의 context이 상위 순위에 위치했으면 높은 점수를 받는다.**
- 상위 5개의 검색 결과 중 1번째, 3번째, 4번째 문서가 관련이 있다고 가정
    ```bash
    Precision@1 = 1/1 = 1.0    # True positive@1/(True positive@1 + False positive@1).  1/1(1번 문서 계산 시에는 1개 문서만 있으므로 분모가 1이 된다.)
    Precision@2 = 1/2 = 0.5     
    Precision@3 = 2/3 ≈ 0.67    
    Precision@4 = 3/4 = 0.75
    Precision@5 = 3/5 = 0.6
    ```
- vk의 값
    - 1번째: $v_1 = 1$
    - 2번째: $v_2 = 0$
    - 3번째: $v_3 = 1$
    - 4번째: $v_4 = 1$
    - 5번째: $v_5 = 0$

- Context Precision@5
$$
\text{Context\;Precision@5} = \frac{(1.0 \times 1) + (0.5 \times 0) + (0.67 \times 1) + (0.75 \times 1) + (0.6 \times 0)}{3} = \frac{1.0 + 0 + 0.67 + 0.75 + 0}{3} ≈ 0.807
$$

# GUIDE
- 예를 들어, 상위 5개의 검색 결과 중 1번째, 3번째, 4번째 문서가 관련이 있다고 가정하면:
```bash
Precision@1 = 1/1 = 1.0     # 1등 -> 1/1  --> 1 등이 관련 없는 것이면 전체 평균 점수가 확 낮아 질 수있다.
Precision@2 = 1/2 = 0.5     # 2등 -> 1/2  (1, 2등 두개 중에 하나만 관련(1번)이 있다.)
Precision@3 = 2/3 ≈ 0.67    # 3등 -> 2/3  (1, 2, 3 등 세개 중에 두개만 관련(1,3번)이 있다.)
Precision@4 = 3/4 = 0.75
Precision@5 = 3/5 = 0.
```
왜 분모가 1, 2, 3, 4, 5 -> 그 순위에는 상위순위와 자신밖에 없으므로 그 개수가 분모가됨. (3등은 1, 2, 3 등 세개 문서로.)

### Context Recall (문맥 재현률)
- 검색된 문서(context)가 얼마나 정답(ground-truth)의 정보를 포함있는 지 평가하는 지표
- 점수 범위: 0~1 (1에 가까울수록 좋음)
- **정답(ground truth)의 각 주장(claim)이 검색된 context와 얼마나 일치**하는지 계산함.

#### 평가방법
1. 정답에서 주장(claim)들을 생성(추출)한다.
    - 예) 
        - **정답**: 한국의 수도는 서울이고 인구수는 5000만명이다. 
        - **주장(claim)**: 
            1. 한국의 수도는 서울이다.
            2. 인구수는 5000만명이다.
2. 각 주장(claim)의 정보를 검색된 contexts에서 찾을 수 있는지 판별한다. 이를 바탕으로 context recall 점수를 계산한다.
    - 예)
        - context: 한국은 동아시아에 위치하고 있는 나라다. 한국의 수도는 서울이다.
        - 위 context에서 추론 가능한 주장: 
            - 한국의 수도는 서울이다. -> context에서 찾을 수 있다.
            - 한국의 인구는 5000만명이다. -> context에서 찾을 수 없다.
3. **Context Recall Score** 를 계산한다. 총 주장 수 중에서 context로 부터 찾을 수 있는 주장의 개수.
    - 예)
        - Context Recall Score = $\cfrac{1}{2} = 0.5$ (두 개의 주장 중 한 개의 주장만 context에서 찾을 수 있다.)

- 공식
    $$
    \text{Context Recall Score}\;=\;\cfrac{\text{GT의\;주장\;중\;주어진\;context\;에서\;찾을\;수\;있는\;주장의\;개수}}{\text{GT의\;총\;주장\;개수}}
    $$ 

# RAGAS 평가 실습

In [ ]:
# !uv pip install ragas rapidfuzz
# 설치 후 커널 재시작

In [2]:
from langchain_text_splitters import MarkdownHeaderTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

from langchain_openai import ChatOpenAI
from langchain_qdrant import FastEmbedSparse, QdrantVectorStore, RetrievalMode
from qdrant_client import QdrantClient, models
from qdrant_client.models import Distance, SparseVectorParams, VectorParams
from langchain_openai import OpenAIEmbeddings

from dotenv import load_dotenv

load_dotenv()


True

In [3]:
# ##############################################################
# 데이터 준비
##############################################################

def load_and_split_olympic_data(file_path="data/olympic_wiki.md"):
    with open(file_path, "r", encoding="utf-8") as fr:
        olympic_text = fr.read()

    # Split
    splitter = MarkdownHeaderTextSplitter(
        headers_to_split_on=[
            ("#", "Header 1"),
            ("##", "Header 2"),
            ("###", "Header 3"),
        ],
        # strip_headers=False, # 문서에 header 포함 여부(default: True - 제거)
    )

    return splitter.split_text(olympic_text)

In [4]:
#################################################################
# Vector DB 연결
# retriever 생성
#################################################################

def get_vectorstore(collection_name: str = "olympic_info_wiki"):


    dense_embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

    client = QdrantClient(host="localhost", port=6333)

    # 컬렉션 삭제
    if client.collection_exists(collection_name):
        result = client.delete_collection(collection_name=collection_name)

    # 컬렉션 생성
    client.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(size=3072, distance=Distance.COSINE),
      
    )

    vectorstore = QdrantVectorStore(
        client=client,
        collection_name=collection_name,    
        embedding=dense_embeddings
    )
    
    ######################################
    # Document들 추가
    ######################################
    documents = load_and_split_olympic_data()
    vectorstore.add_documents(documents=documents)

    return vectorstore

# 벡터스토어를 리트리버로 만들어줌
def get_retriever(vectorstore, k: int = 5):
    retriever = vectorstore.as_retriever(
        search_kwargs={"k": k}
    )
    return retriever

In [5]:
vectorstore = get_vectorstore()

retriever = get_retriever(vectorstore)
retriever

VectorStoreRetriever(tags=['QdrantVectorStore', 'OpenAIEmbeddings'], vectorstore=<langchain_qdrant.qdrant.QdrantVectorStore object at 0x000002DC3E927DA0>, search_kwargs={'k': 5})

In [6]:
################################################################################
# 평가할 RAG Chain
################################################################################

from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough 
from langchain_core.output_parsers import StrOutputParser
from langchain_core.documents import Document


vectorstore = get_vectorstore()
retriever = get_retriever(vectorstore)

prompt_txt = """# Instruction:
당신은 정보제공을 목적으로하는 유능한 AI Assistant 입니다.
주어진 context의 내용을 기반으로 질문에 답변을 합니다.
Context에 질문에 대한 명확한 정보가 있는 경우 그것을 바탕으로 답변을 합니다.
Context에 질문에 대한 명확한 정보가 없는 경우 "정보가 부족해 답을 할 수없습니다." 라고 답합니다.
절대 추측이나 일반 상식을 바탕으로 답을 하거나 Context 없는 내용을 만들어서 답변해서는 안됩니다.

# Context:
{context}

# 질문:
{query}
"""
prompt = ChatPromptTemplate.from_template(
    template=prompt_txt
)


def format_docs(documents:list)->str:
    """
    VectorStore에 조회한 문서들에서 내용(page_content)만 추출해서 str으로 합쳐서 반환.
    VectorStore의 검색결과인 List[Document]를 받아서 Document들에서 page_content의 내용만 추출한다.
    
    Args:
        documents(list[Document]): [Document(..), Document(...), ..]}
    Returns:
        str: 각 문서의 내용을 "\n\n"으로 연결한 string
    """
    return "\n\n".join(doc.page_content for doc in documents)

model = ChatOpenAI(model="gpt-5-mini")
parser = StrOutputParser()

# chain = {
#     "context":retriever | format_docs,
#     "query":RunnablePassthrough()
# } | prompt | model | parser

from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from operator import itemgetter

def format_doc_list(docs_dict:dict) -> list:
    #list[Document]->list[str] 
    # 문서내용만 추출해서(Document.page_content)만 추출한 리스트
    return [doc.page_content for doc in docs_dict['context']]

# RAG 평가를 위해서 "답변", "검색한 문서" 둘이 출력되도록 변경
# dict | dict -> dict
# Runnablepassthrough() | dict | dict ==> RunnableSequence
chain = RunnablePassthrough() | {
    "context":retriever,
    "query":RunnablePassthrough()
} | {
    # "response": RunnableLambda(lambda x : x['context']) | format_docs | prompt | model | parser,
    "response": prompt | model | parser,
    "retrieved_context": format_doc_list, # RAGAS 평가시 context -> List[str]
}


In [7]:
res = chain.invoke("1회 올림픽은 언제 어디서 열렸지")

In [52]:
res.keys()

dict_keys(['response', 'retrieved_context'])

In [53]:
print(res['response'])

제1회 근대 올림픽은 1896년에 그리스 아테네에서 열렸습니다.


In [61]:
import pprint
pprint.pprint(res['retrieved_context'])

['고대의 올림픽 경기(올림피아 경기)는 고대 그리스의 여러 도시 국가의 대표선수들이 모여 벌인 일련의 시합이었으며, 육상 경기가 주 '
 '종목이지만 격투기와 전차 경기도 열렸다. 그리고 패배하면 죽기도 하였다. 고대 올림픽의 유래는 수수께끼로 남아있다. 잘 알려진 신화로는 '
 "헤라클레스와 그의 아버지인 제우스가 올림픽의 창시자였다는 것이다. 전설에 따르면 이 경기를 최초로 '올림픽'이라고 부르고, 4년마다 "
 '대회를 개최하는 관례를 만든 사람이 헤라클레스라고 한다. 어떤 전설에서는 헤라클레스가 이른바 헤라클레스의 12업을 달성한 뒤에 제우스를 '
 '기리고자 올림픽 경기장을 지었다고 한다. 경기장이 완성되자 헤라클레스는 일직선으로 200 걸음을 걸었으며, 이 거리를 "스타디온"이라 '
 "불렀는데, 후에 이것이 길이 단위인 '스타디온'(그리스어: στάδιον → 라틴어: 영어: stadium)이 되었다. 또 다른 설로는 "
 "'올림픽 휴전'(그리스어: ἐκεχειρία 에케케이리아[*])이라는 고대 그리스의 관념이 최초의 올림피아 경기와 관련이 있다고 한다. "
 "'올림픽 휴전'이란 어느 도시 국가라도 올림피아 경기 기간 중에 다른 나라를 침범하면 그에 대한 응징을 받을 수 있다는 뜻으로, "
 '"올림픽 기간에는 전쟁하지 말 것"으로 요약할 수 있다.  \n'
 '고대 올림피아 경기가 처음 열린 시점은 보통 기원전 776년으로 인정되고 있는데, 이 연대는 그리스 올림피아에서 발견된 비문에 근거를 둔 '
 '것이다. 이 비문의 내용은 달리기 경주 승자 목록이며 기원전 776년부터 4년 이후 올림피아 경기 마다의 기록이 남겨져 있다. 고대 '
 '올림픽의 종목으로는 육상, 5종 경기(원반던지기, 창던지기, 달리기, 레슬링, 멀리뛰기), 복싱, 레슬링, 승마 경기가 있었다. 전설에 '
 '따르면 엘리스의 코로이보스가 최초로 올림피아 경기에서 우승한 사람이라고 한다.  \n'
 '고대 올림피아 경기는 근본적으로 종교적인 중요성을 띄고 있었는데, 스포

# RAGAS 를 이용해 평가를 위한 합성 데이터 셋 만들기

- 평가 데이터셋 구성
  - `user_input`: 사용자 질문
  - `retrieved_contexts`: Vectorstore에서 검색한 context
  - `response`: LLM의 응답
  - `reference`: 정답

## TestsetGenerator
- **문서(retrieved_contexts)를 기준**으로 **질문**, **정답** 을 생성한다.
- 평가할 LLM으로 생성된 질문을 넣어 답변을 추출하여 데이터셋을 구성한다.

In [8]:
# 주피터노트북 환경에서 비동기적 처리 위해
# script(.py) 로 작성할 경우는 필요 없다.

import nest_asyncio
nest_asyncio.apply()


In [9]:
#########################################################
# 데이터셋을 생성할 때 사용할 context를 추출 - sampling
#########################################################
client = QdrantClient(host="localhost", port=6333)
COLLECTION_NAME = "olympic_info_wiki"

# 전체 데이터를 다 조회해서 그 중 랜덤하게 K개만 sampling
info = client.get_collection(COLLECTION_NAME)
# info.points_count # 저장된 데이터 개수
results, next_id = client.scroll(
    collection_name=COLLECTION_NAME,
    limit=info.points_count
)
len(results)
# sampling
import random
sample_dataset = random.sample(results, 5) # 리스트에서 랜덤하게 K개를 추출

# 문서 내용만 추출
docs = [point.payload['page_content'] for point in sample_dataset]
docs

["프로 NHL선수들은 1998년부터 아이스 하키종목에 출전할 수 있게 되었다. (나가노 올림픽 결승전 러시아 vs 체코).  \n영국 명문 공립 학교의 이념은 쿠베르탱에게 큰 영향을 끼쳤다. 영국 공립 학교는 스포츠를 교육의 중요한 부분이라 생각해서 '건전한 신체에 건전한 정신을'이라는 의미를 가진 라틴어 mens sana in corpore sano를 표어로 삼았다. 이 이념에 의하면 신사들은 특정한 분야에서만 우수해서는 안되고 모든 분야에서 고르게 잘해야 하고, 공정한 결과에는 승복해야 하며, 연습이나 훈련은 속이는 것과 마찬가지로 여겼다. 전문적으로 스포츠를 연습한 사람은 취미로 연습한 사람에 비해 공평하지 않다고 생각한 것이다.  \n현대 올림픽에서는 프로 선수의 참가 불허가 많은 분쟁을 가져왔다. 1912년 하계 올림픽의 근대 5종 경기와 10종 경기에서 우승한 짐 소프는 올림픽에 나가기 전에 준프로야구선수로 활동했다는 게 나중에 밝혀져 메달이 박탈되었다. 소프는 후에 동정적 여론의 힘을 업고 1983년에 메달을 돌려받게 된다. 1936년 동계 올림픽 때 스위스와 오스트리아 스키선수들은 돈을 벌기 위해 스포츠를 했는데 이러한 행동이 아마추어 정신에 위배된다고 결정되어 그들은 스키종목에 참가할 수 없었다.  \n20세기에 이르러서 계급구조가 붕괴되면서 이른바 귀족적인 신사라는 아마추어 선수에 대한 정의는 시대에 뒤처지는 말이 되게 된다. 일부 국가들은 '정식 아마추어 선수'를 '키워서' 순수한 아마추어 정신을 벗어나고 있었고, 자신이 내는 비용으로 연습하는 선수들의 불리함에 대한 목소리가 나오기 시작했다. 하지만 IOC는 아마추어 정신에 관한 입장을 고수했다. 1970년대 초에는 아마추어 정신이 올림픽헌장에서 폐지되어야 한다는 말이 나오기 시작했다. 결국 프로선수들의 출전은 국제경기연맹(IF)에서 결정짓도록 되었다. 2008년 기준으로 아마추어 선수만 출전하고 있는 올림픽 종목은 복싱이 유일하며 남자 축구에서는 나이가 23세 이상인 선수를 3명까지만 

In [22]:
sample_dataset[0]

Record(id='d1cdb6f0-37f0-446e-b76f-cadbe3ded449', payload={'page_content': '20세기 초반, 많은 운동 선수들은 기록향상을 위해 약물을 복용하기 시작했다. 예를 들어 1904년 하계 올림픽 마라톤에서 우승한 미국 선수 토머스 J. 힉스는 코치에게서 스트리크닌과 브랜디를 받았다. 올림픽에서 약물을 과다 복용으로 사망한 사례도 한 번 있었다. 1960년 로마 대회 때 사이클 개인도로 경기 중에 덴마크 선수인 크누드 에네마르크 옌센이 자전거에서 떨어져서 사망했다. 검시관들의 조사에 의하면 그의 죽음의 원인은 암페타민 과다 복용이라고 했다. 이에 1960년대 중반부터 각 경기 연맹은 약물 복용을 금지하기 시작했으며 1967년에는 IOC도 약물 복용 금지에 동참했다.  \n올림픽에서 약물 복용 양성 반응이 나와서 메달을 박탈당한 첫 번째 사례로는 1968년 하계 올림픽의 근대 5종 경기에 출전해 동메달을 딴 한스 군나르 리렌바르가 있다. 그는 경기 후 도핑검사 결과 알코올을 복용한 것으로 확인되어 메달을 박탈당했다. 도핑 양성 반응으로 메달을 박탈당한 것으로 가장 유명한 사람은 1988년 하계 올림픽 육상 100m 경기에서 금메달을 땄으나 도핑 검사 결과 스타노졸롤을 복용한 것으로 확인돼 금메달을 박탈당한 캐나다 선수인 벤 존슨이 있다. 이에 따라 금메달은 2위를 했던 칼 루이스가 대신 받았다.  \n1990년대 후반, 여러 뜻있는 사람들이 도핑과의 전쟁을 선포하면서 1999년에 세계반도핑기구(WADA)를 설립한다. 2000년 하계 올림픽과 2002년 동계 올림픽 때는 약물 양성 반응을 보인 선수들이 급격히 증가했고, 역도와 크로스컨트리에서는 몇몇 선수들이 도핑 테스트에 걸려서 실격되기도 했다. 2006년 동계 올림픽 때는 메달리스트 한 명이 양성반응을 보여 메달을 반납해야 했다. IOC가 만든 약물 반응 판정(현재 올림픽 도핑테스트의 기준이 됨)은 인정을 받게 되었고 이제는 다른 경기 연맹에서도 벤치마킹을 할

In [10]:
from ragas.testset import TestsetGenerator
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
# gpt-5 버전은 사용할 수 없다. (temperature 설정 문제가 있다.)
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1")) # Langchain모델을 RAGAS에서 사용할 수 있도록 변환(Wrapping)
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model="text-embedding-3-large"))

generator = TestsetGenerator(
    llm=generator_llm,
    embedding_model=generator_embeddings,
    llm_context="데이터셋은 반드시 한국어로 작성한다. 데이터셋은 JSON 문법을 꼭 지켜서 작성한다. 특히 구두점은 꼭 지켜야 한다. Document에 JSON 문법에 맞지 않는 표현이 있으면 반드시 수정해서 처리한다."
    # 질문/답변 생성할 때 llm에 전달할 system prompt를 설정.
)

C:\Users\Playdata\AppData\Local\Temp\ipykernel_16848\2336467797.py:7: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1")) # Langchain모델을 RAGAS에서 사용할 수 있도록 변환(Wrapping)
C:\Users\Playdata\AppData\Local\Temp\ipykernel_16848\2336467797.py:8: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.embeddings import OpenAIEmbeddings, GoogleEmbeddings, HuggingFaceEmbeddings
  generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model="text-embedding-3-large"))


In [11]:
testset = generator.generate_with_chunks(
    docs, testset_size=10 # context 내용, 테스트데이터셋 몇개를 만들지.
)

Applying SummaryExtractor:   0%|          | 0/5 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/5 [00:00<?, ?it/s]

Node 311a8ec9-8d49-4f3d-99b5-06f85d9fd9ba does not have a summary. Skipping filtering.
Node 3316e9d4-3035-46d8-845c-fddf6a1fb545 does not have a summary. Skipping filtering.
Node 5aae39dc-dbd5-459f-b11b-5fb40e79b413 does not have a summary. Skipping filtering.
Node a9369010-ab0d-48ec-bfea-1f2fa71fc2a6 does not have a summary. Skipping filtering.
Node cf667b85-dfa9-4e81-8956-2d2c4946c8fe does not have a summary. Skipping filtering.


Applying EmbeddingExtractor:   0%|          | 0/5 [00:00<?, ?it/s]

Applying ThemesExtractor:   0%|          | 0/5 [00:00<?, ?it/s]

...

Skipping multi_hop_abstract_query_synthesizer due to unexpected error: No relationships match the provided condition. Cannot form clusters.


Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/2 [00:00<?, ?it/s]

Exception in callback Task.__step()
handle: <Handle Task.__step()>
Traceback (most recent call last):
  File "C:\Users\Playdata\AppData\Roaming\uv\python\cpython-3.12.12-windows-x86_64-none\Lib\asyncio\events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
RuntimeError: cannot enter context: <_contextvars.Context object at 0x000002DBA5F2C180> is already entered
Exception in callback Task.__step()
handle: <Handle Task.__step()>
Traceback (most recent call last):
  File "C:\Users\Playdata\AppData\Roaming\uv\python\cpython-3.12.12-windows-x86_64-none\Lib\asyncio\events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
RuntimeError: cannot enter context: <_contextvars.Context object at 0x000002DBA5F2C180> is already entered
Exception in callback Task.__step()
handle: <Handle Task.__step()>
Traceback (most recent call last):
  File "C:\Users\Playdata\AppData\Roaming\uv\python\cpython-3.12.12-windows-x86_64-none\Lib\asyncio\events.py", line 88,

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

In [28]:
print(type(testset))
testset

<class 'ragas.testset.synthesizers.testset_schema.Testset'>


Testset(samples=[TestsetSample(eval_sample=SingleTurnSample(user_input='1904년 하계 올림픽 마라톤에서 우승한 선수는 약물 복용과 관련하여 어떤 일이 있었나요?', retrieved_contexts=None, reference_contexts=['20세기 초반, 많은 운동 선수들은 기록향상을 위해 약물을 복용하기 시작했다. 예를 들어 1904년 하계 올림픽 마라톤에서 우승한 미국 선수 토머스 J. 힉스는 코치에게서 스트리크닌과 브랜디를 받았다. 올림픽에서 약물을 과다 복용으로 사망한 사례도 한 번 있었다. 1960년 로마 대회 때 사이클 개인도로 경기 중에 덴마크 선수인 크누드 에네마르크 옌센이 자전거에서 떨어져서 사망했다. 검시관들의 조사에 의하면 그의 죽음의 원인은 암페타민 과다 복용이라고 했다. 이에 1960년대 중반부터 각 경기 연맹은 약물 복용을 금지하기 시작했으며 1967년에는 IOC도 약물 복용 금지에 동참했다.  \n올림픽에서 약물 복용 양성 반응이 나와서 메달을 박탈당한 첫 번째 사례로는 1968년 하계 올림픽의 근대 5종 경기에 출전해 동메달을 딴 한스 군나르 리렌바르가 있다. 그는 경기 후 도핑검사 결과 알코올을 복용한 것으로 확인되어 메달을 박탈당했다. 도핑 양성 반응으로 메달을 박탈당한 것으로 가장 유명한 사람은 1988년 하계 올림픽 육상 100m 경기에서 금메달을 땄으나 도핑 검사 결과 스타노졸롤을 복용한 것으로 확인돼 금메달을 박탈당한 캐나다 선수인 벤 존슨이 있다. 이에 따라 금메달은 2위를 했던 칼 루이스가 대신 받았다.  \n1990년대 후반, 여러 뜻있는 사람들이 도핑과의 전쟁을 선포하면서 1999년에 세계반도핑기구(WADA)를 설립한다. 2000년 하계 올림픽과 2002년 동계 올림픽 때는 약물 양성 반응을 보인 선수들이 급격히 증가했고, 역도와 크로스컨트리에서는 몇몇 선수들이 도핑 테스트에 걸려서 실격되기도 했다. 2006년 동계 올림픽 때는 메달리스트 한 명이 

In [12]:
sample1 = testset.samples[0].eval_sample
print(sample1.user_input)           # 사용자 질문
print(sample1.reference_contexts)   # Vector DB의 Context (이것을 바탕으로 user_input과 reference를 생성)
print(sample1.retrieved_contexts)   # RAG pipeline이 검색한 문서
print(sample1.reference)            # 정답(ground truth)
print(sample1.response)             # LLM 응답 (정답추정값)
# 평가시 필요한 속성: user_input, retrieved_contexts, reference, response
# retrieved_context, response -> 평가할 RAG Chain으로부터 가져와야 한다.

체코 하키 올림픽 결승 언제?
["프로 NHL선수들은 1998년부터 아이스 하키종목에 출전할 수 있게 되었다. (나가노 올림픽 결승전 러시아 vs 체코).  \n영국 명문 공립 학교의 이념은 쿠베르탱에게 큰 영향을 끼쳤다. 영국 공립 학교는 스포츠를 교육의 중요한 부분이라 생각해서 '건전한 신체에 건전한 정신을'이라는 의미를 가진 라틴어 mens sana in corpore sano를 표어로 삼았다. 이 이념에 의하면 신사들은 특정한 분야에서만 우수해서는 안되고 모든 분야에서 고르게 잘해야 하고, 공정한 결과에는 승복해야 하며, 연습이나 훈련은 속이는 것과 마찬가지로 여겼다. 전문적으로 스포츠를 연습한 사람은 취미로 연습한 사람에 비해 공평하지 않다고 생각한 것이다.  \n현대 올림픽에서는 프로 선수의 참가 불허가 많은 분쟁을 가져왔다. 1912년 하계 올림픽의 근대 5종 경기와 10종 경기에서 우승한 짐 소프는 올림픽에 나가기 전에 준프로야구선수로 활동했다는 게 나중에 밝혀져 메달이 박탈되었다. 소프는 후에 동정적 여론의 힘을 업고 1983년에 메달을 돌려받게 된다. 1936년 동계 올림픽 때 스위스와 오스트리아 스키선수들은 돈을 벌기 위해 스포츠를 했는데 이러한 행동이 아마추어 정신에 위배된다고 결정되어 그들은 스키종목에 참가할 수 없었다.  \n20세기에 이르러서 계급구조가 붕괴되면서 이른바 귀족적인 신사라는 아마추어 선수에 대한 정의는 시대에 뒤처지는 말이 되게 된다. 일부 국가들은 '정식 아마추어 선수'를 '키워서' 순수한 아마추어 정신을 벗어나고 있었고, 자신이 내는 비용으로 연습하는 선수들의 불리함에 대한 목소리가 나오기 시작했다. 하지만 IOC는 아마추어 정신에 관한 입장을 고수했다. 1970년대 초에는 아마추어 정신이 올림픽헌장에서 폐지되어야 한다는 말이 나오기 시작했다. 결국 프로선수들의 출전은 국제경기연맹(IF)에서 결정짓도록 되었다. 2008년 기준으로 아마추어 선수만 출전하고 있는 올림픽 종목은 복싱이 유일하며 남자 축구에서는 나이가 2

In [13]:
# Testset을 pandas DataFrame을 변환 -> to_pandas(), to_xxxx() 를 통해서 다른 구조로 변환이 가능.
eval_df = testset.to_pandas()
eval_df

,user_input,reference_contexts,reference,persona_name,query_style,query_length,synthesizer_name
0,체코 하키 올림픽 결승 언제?,[프로 NHL선수들은 1998년부터 아이스 하키종목에 출전할 수 있게 되었다. (나...,체코는 1998년 나가노 올림픽 결승전에서 러시아와 아이스 하키 경기를 치렀다.,Inclusive Sports Educator,MISSPELLED,SHORT,single_hop_specific_query_synthesizer
1,런던 올림픽과 패럴림픽의 시작은 어떻게 연결되어 있나요?,[패럴림픽(Paralympic)은 신체·감각 장애가 있는운동 선수가 참가하는 국제 ...,1948년 런던 올림픽과 동시에 루드비히 구트만 경은 제2차 세계대전에 참전한 군인...,Sports Historian,WEB_SEARCH_LIKE,MEDIUM,single_hop_specific_query_synthesizer
2,IOC 뭐하는데? 올림픽 개최지 어떻게 정하는지 알려줘.,[올림픽 개최지는 해당 올림픽 개최 7년 전에 IOC 위원들의 투표로 결정된다. 개...,올림픽 개최지는 해당 올림픽 개최 7년 전에 IOC 위원들의 투표로 결정된다. 개최...,International Sports Event Planner,POOR_GRAMMAR,MEDIUM,single_hop_specific_query_synthesizer
3,헝가리와 관련된 올림픽 보이콧 사례는?,"[올림픽에서 첫 번째 보이콧은 1956년 하계 올림픽에서 시작되었다. 네덜란드, 스...","1956년 하계 올림픽에서 네덜란드, 스페인, 스위스는 소련의 헝가리 침공에 항의해...",Sports Historian,WEB_SEARCH_LIKE,SHORT,single_hop_specific_query_synthesizer
4,아테네 올림픽에서 메달 말고 뭐 줬나요?,"[개인 혹은 팀으로 경기에 출전해서 1위, 2위, 3위를 한 선수는 메달을 받는다....","아테네에서 열린 2004년 하계 올림픽 때는 1, 2, 3위 선수에게 메달과 함께 ...",Inclusive Sports Educator,MISSPELLED,SHORT,single_hop_specific_query_synthesizer
5,1988년 하계 올림픽과 패럴림픽 개최 도시는 어떤 관계가 있나요?,[<1-hop>\n\n패럴림픽(Paralympic)은 신체·감각 장애가 있는운동 선...,1988년 하계 올림픽부터 하계 올림픽을 개최한 도시는 패럴림픽도 같이 개최하기로 ...,NaN,NaN,NaN,multi_hop_specific_query_synthesizer
6,"패럴림픽이 1960년 하계 올림픽과 어떤 관련이 있으며, 이 대회가 패럴림픽 역사에...",[<1-hop>\n\n패럴림픽(Paralympic)은 신체·감각 장애가 있는운동 선...,1960년 하계 올림픽이 열린 로마에서 구트만은 400명의 선수들을 'Paralle...,NaN,NaN,NaN,multi_hop_specific_query_synthesizer
7,"1960년 하계 올림픽과 관련하여 패럴림픽이 어떻게 시작되었으며, 이 대회가 이후 ...",[<1-hop>\n\n패럴림픽(Paralympic)은 신체·감각 장애가 있는운동 선...,패럴림픽은 1948년 루드비히 구트만 경이 제2차 세계대전에 참전한 군인들의 사회 ...,NaN,NaN,NaN,multi_hop_specific_query_synthesizer
8,"1988년 하계 올림픽과 관련하여, 패럴림픽이 하계 올림픽과 같은 도시에서 개최되도...",[<1-hop>\n\n패럴림픽(Paralympic)은 신체·감각 장애가 있는운동 선...,"패럴림픽은 신체 및 감각 장애가 있는 운동 선수가 참가하는 국제 스포츠 대회로, 1...",NaN,NaN,NaN,multi_hop_specific_query_synthesizer
9,"1904년 하계 올림픽에서 메달 수여 방식이 어떻게 바뀌었고, 그 이전 1896년 ...","[<1-hop>\n\n개인 혹은 팀으로 경기에 출전해서 1위, 2위, 3위를 한 선...",1904년 하계 올림픽부터 현재와 같은 메달 수여 방식이 시작되었어요. 이 방식에서...,NaN,NaN,NaN,multi_hop_specific_query_synthesizer


In [14]:
res = chain.invoke(eval_df['user_input'][0])
res['response']

'1998년 나가노 올림픽 결승전입니다. (결승: 러시아 vs 체코)'

In [15]:
# LLM 응답들을 저장할 list
response_list = []
# Chain 이 반환한 context들을 저장할 list
retrieved_context_list = []

for user_input in eval_df['user_input']:
    res = chain.invoke(user_input)
    response_list.append(res['response'])
    retrieved_context_list.append(res['retrieved_context'])

In [16]:
retrieved_context_list

[['올림픽 개최지는 해당 올림픽 개최 7년 전에 IOC 위원들의 투표로 결정된다. 개최지 선정에는 약 2년이 걸린다. 유치를 희망하는 도시는 우선 자국의 올림픽 위원회에 신청을 해야 한다. 만약 한 국가에서 두 도시 이상이 유치를 희망한다면, 한 국가당 한 도시만 후보가 될 수 있다는 규칙에 따라 내부적으로 후보 도시를 결정해야 한다. 후보 도시가 결정되면 후보 도시가 소속된 국가의 올림픽 위원회는 IOC에 개최 신청을 하고, 신청 후에는 올림픽 개최에 대한 질의 응답서를 보내야 한다. 이 질의응답서에서 신청한 도시는 올림픽 헌장을 준수하며 IOC 상임이사회에 의한 다른 규정들을 지킬 것이라는 확신을 주어야 한다. 이 질의응답서는 전문가들이 검토하여 신청 도시들의 잠재성과 계획을 평가한다. 이 전문적인 평가를 바탕으로 IOC 상임이사회에서는 신청도시 중에서 후보도시를 고른다.  \n후보도시로 선택되면 그 도시들은 IOC에 보내는 후보도시에 관한 문서에 그들의 계획을 더욱 상세하고 방대한 양으로 적어서 보내야 한다. 평가조사단들이 이 후보도시들을 평가한다. 평가조사단은 후보도시들을 방문해서 지역 관계자들과 회견을 갖고 경기장 시설을 세심하게 조사한 뒤 개최지 투표를 하기 한달전에 조사를 바탕으로 한 공식 보고를 한다. 회견을 하는 동안에도 후보도시들은 자신들이 올림픽을 개최하는 데 충분한 자금이 조달될 수 있는지 등을 입증할 수 있어야 한다. 평가조사단의 업무가 끝나면 후보지의 국가 위원들은 IOC 정기총회에 참석한다. 이 총회에서 IOC 위원들은 올림픽 개최지를 선정하게 되며 후보지의 국가에 소속된 위원들은 자국의 후보지가 탈락하지 않는 이상 투표를 할 수 없다. 투표가 끝난후에 개최지로 선정된 곳의 유치위원회가 IOC와 개최도시 계약서에 서명을 하면 공식적으로 올림픽 개최도시(개최국)으로 인정된다.  \n2016년까지 올림픽은 23개국 44개 도시에서 열렸으며 유럽과 북아메리카대륙 이외의 대륙에서는 고작 8번 밖에 개최하지 못했다. 1988년 하계 올림픽이

In [17]:
response_list

['1998년 (나가노 올림픽 결승전)',
 '1948년 런던 올림픽과 동시에 루드비히 구트만 경이 제2차 세계대전 참전 군인의 사회 복귀를 위해 몇몇 병원들을 연합해 휠체어 선수들을 대상으로 여러 경기를 열었습니다. 이 대회(세계 휠체어·신체부자유자대회)가 발전해 1960년 로마 올림픽 때 400명의 선수가 참가한 "Parallel Olympics"(초기 패럴림픽)가 되었고, 이후 패럴림픽은 하계 올림픽이 열린 해에 함께 열리게 되었습니다. (또한 1988년 서울 올림픽부터는 개최 도시가 패럴림픽도 같이 개최하도록 정해졌습니다.)',
 '간단히 정리하면 다음과 같습니다.\n\n1) IOC(국제 올림픽 위원회)는 뭐하는 곳인가?\n- 올림픽 활동을 통솔하는 단체(의사결정 기구)입니다.\n- 올림픽 개최 도시 선정, 개최 계획 감독, 종목 변경 결정, 스폰서 및 방송권 계약 체결 등의 권한을 가집니다.\n- 조직과 활동은 올림픽 헌장을 따릅니다.\n\n2) 올림픽 개최지는 어떻게 정하나?\n- 개최지는 개최 7년 전에 IOC 위원들의 투표로 결정되며, 선정에는 약 2년이 걸립니다.\n- 유치 희망 도시는 먼저 자국의 국가 올림픽 위원회(NOC)에 신청해야 하고, 한 나라에서는 한 도시만 후보가 될 수 있습니다.\n- 신청 후 질의응답서(문서)를 제출하면 전문가들이 이를 검토하여 잠재성과 계획을 평가합니다.\n- IOC 상임이사회(집행부)가 신청 도시들 중 후보도시를 선정합니다.\n- 후보도시는 더 상세한 계획서를 제출하고 평가조사단이 방문·조사하여 개최 가능성, 경기장 시설, 자금 조달 능력 등을 점검합니다. 평가조사단은 투표 한 달 전에 공식 보고서를 제출합니다.\n- 평가가 끝나면 후보 도시의 국가 위원들이 IOC 총회에 참석하고, IOC 위원들이 개최지를 선정(투표)합니다. 후보지의 국가에 소속된 IOC 위원들은 자국 후보가 남아 있는 한 투표할 수 없습니다.\n- 투표로 선정된 도시의 유치위원회가 IOC와 개최도시 계약서에 서명하면 공식 개최지로 인정됩니

In [18]:
# eval_df에 추가
eval_df['response'] = response_list
eval_df['retrieved_contexts'] = retrieved_context_list
eval_df.head()

,user_input,reference_contexts,reference,persona_name,query_style,query_length,synthesizer_name,response,retrieved_contexts
0,체코 하키 올림픽 결승 언제?,[프로 NHL선수들은 1998년부터 아이스 하키종목에 출전할 수 있게 되었다. (나...,체코는 1998년 나가노 올림픽 결승전에서 러시아와 아이스 하키 경기를 치렀다.,Inclusive Sports Educator,MISSPELLED,SHORT,single_hop_specific_query_synthesizer,1998년 (나가노 올림픽 결승전),[올림픽 개최지는 해당 올림픽 개최 7년 전에 IOC 위원들의 투표로 결정된다. 개...
1,런던 올림픽과 패럴림픽의 시작은 어떻게 연결되어 있나요?,[패럴림픽(Paralympic)은 신체·감각 장애가 있는운동 선수가 참가하는 국제 ...,1948년 런던 올림픽과 동시에 루드비히 구트만 경은 제2차 세계대전에 참전한 군인...,Sports Historian,WEB_SEARCH_LIKE,MEDIUM,single_hop_specific_query_synthesizer,1948년 런던 올림픽과 동시에 루드비히 구트만 경이 제2차 세계대전 참전 군인의 ...,[패럴림픽(Paralympic)은 신체·감각 장애가 있는운동 선수가 참가하는 국제 ...
2,IOC 뭐하는데? 올림픽 개최지 어떻게 정하는지 알려줘.,[올림픽 개최지는 해당 올림픽 개최 7년 전에 IOC 위원들의 투표로 결정된다. 개...,올림픽 개최지는 해당 올림픽 개최 7년 전에 IOC 위원들의 투표로 결정된다. 개최...,International Sports Event Planner,POOR_GRAMMAR,MEDIUM,single_hop_specific_query_synthesizer,간단히 정리하면 다음과 같습니다.\n\n1) IOC(국제 올림픽 위원회)는 뭐하는 ...,"[올림픽 활동이란 많은 수의 국가, 국제 경기 연맹과 협회 • 미디어 파트너를 맺기..."
3,헝가리와 관련된 올림픽 보이콧 사례는?,"[올림픽에서 첫 번째 보이콧은 1956년 하계 올림픽에서 시작되었다. 네덜란드, 스...","1956년 하계 올림픽에서 네덜란드, 스페인, 스위스는 소련의 헝가리 침공에 항의해...",Sports Historian,WEB_SEARCH_LIKE,SHORT,single_hop_specific_query_synthesizer,"1956년 하계 올림픽에서 소련의 헝가리 침공에 항의하여 네덜란드, 스페인, 스위스...","[올림픽에서 첫 번째 보이콧은 1956년 하계 올림픽에서 시작되었다. 네덜란드, 스..."
4,아테네 올림픽에서 메달 말고 뭐 줬나요?,"[개인 혹은 팀으로 경기에 출전해서 1위, 2위, 3위를 한 선수는 메달을 받는다....","아테네에서 열린 2004년 하계 올림픽 때는 1, 2, 3위 선수에게 메달과 함께 ...",Inclusive Sports Educator,MISSPELLED,SHORT,single_hop_specific_query_synthesizer,아테네(2004년) 올림픽에서는 메달과 함께 올리브 화환도 수여했습니다.,"[개인 혹은 팀으로 경기에 출전해서 1위, 2위, 3위를 한 선수는 메달을 받는다...."


In [20]:
#######################################################
# EvaluationDataset을 생성 -> RAGAS 평가 데이터셋 타입
#######################################################
from ragas import EvaluationDataset
# from_xxxx() : xxxx타입의 객체를 EvaluationDataset 객체로 변환.
eval_dataset = EvaluationDataset.from_pandas(
    eval_df[["user_input", "retrieved_contexts", "response", "reference"]]
)
eval_dataset

EvaluationDataset(features=['user_input', 'retrieved_contexts', 'response', 'reference'], len=11)

In [26]:
#############################
# 평가
#############################
from ragas.metrics import (
    LLMContextRecall, # Context Recall 평가함수
    LLMContextPrecisionWithReference, # Context Precision
    Faithfulness,
    AnswerRelevancy
)
from ragas import evaluate

eval_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1"))
eval_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model="text-embedding-3-large"))
# 평가할 함수들을 List로 묶어준다.
metrics = [
    LLMContextRecall(llm=eval_llm),
    LLMContextPrecisionWithReference(llm=eval_llm),
    Faithfulness(llm=eval_llm),
    AnswerRelevancy(llm=eval_llm, embeddings=eval_embeddings)
]

# 평가를 진행 - evaluate() 함수 이용
eval_result = evaluate(dataset=eval_dataset, metrics=metrics)

C:\Users\Playdata\AppData\Local\Temp\ipykernel_16848\2890642656.py:4: DeprecationWarning: Importing LLMContextRecall from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import LLMContextRecall
  from ragas.metrics import (
C:\Users\Playdata\AppData\Local\Temp\ipykernel_16848\2890642656.py:4: DeprecationWarning: Importing LLMContextPrecisionWithReference from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import LLMContextPrecisionWithReference
  from ragas.metrics import (
C:\Users\Playdata\AppData\Local\Temp\ipykernel_16848\2890642656.py:4: DeprecationWarning: Importing Faithfulness from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import Faithfulness
  from ragas.metrics import (
C:\User

Evaluating:   0%|          | 0/44 [00:00<?, ?it/s]

LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
Exception in callback Task.__step()
handle: <Handle Task.__step()>
Traceback (most recent call last):
  File "C:\Users\Playdata\AppData\Roaming\uv\python\cpython-3.12.12-windows-x86_64-none\Lib\asyncio\events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
RuntimeError: cannot enter context: <_contextvars.Context object at 0x000002DBA5F2C180> is already entered
Exception in callback Task.__step()
handle: <Handle Task.__step()>
Traceback (most recent call last):
  File "C:\Users\Playdata\AppData\Roaming\uv\python\cpython-3.12.12-windows-x86_64-none\Lib\asyncio\events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
RuntimeError: cannot enter context: <_contextvars.Context object at 0x000002DBA5F2C180> is already entered
Exception in callback Task.__step()
handle: <Handle Task._

In [27]:
eval_result

{'context_recall': 0.9407, 'llm_context_precision_with_reference': 0.8939, 'faithfulness': 0.7500, 'answer_relevancy': 0.5469}

In [28]:
# 개별 Point에 대한 평가 결과.
result_df = eval_result.to_pandas()
result_df


,user_input,retrieved_contexts,response,reference,context_recall,llm_context_precision_with_reference,faithfulness,answer_relevancy
0,체코 하키 올림픽 결승 언제?,[올림픽 개최지는 해당 올림픽 개최 7년 전에 IOC 위원들의 투표로 결정된다. 개...,1998년 (나가노 올림픽 결승전),체코는 1998년 나가노 올림픽 결승전에서 러시아와 아이스 하키 경기를 치렀다.,1.000000,0.500000,NaN,0.472070
1,런던 올림픽과 패럴림픽의 시작은 어떻게 연결되어 있나요?,[패럴림픽(Paralympic)은 신체·감각 장애가 있는운동 선수가 참가하는 국제 ...,1948년 런던 올림픽과 동시에 루드비히 구트만 경이 제2차 세계대전 참전 군인의 ...,1948년 런던 올림픽과 동시에 루드비히 구트만 경은 제2차 세계대전에 참전한 군인...,1.000000,1.000000,NaN,0.731846
2,IOC 뭐하는데? 올림픽 개최지 어떻게 정하는지 알려줘.,"[올림픽 활동이란 많은 수의 국가, 국제 경기 연맹과 협회 • 미디어 파트너를 맺기...",간단히 정리하면 다음과 같습니다.\n\n1) IOC(국제 올림픽 위원회)는 뭐하는 ...,올림픽 개최지는 해당 올림픽 개최 7년 전에 IOC 위원들의 투표로 결정된다. 개최...,1.000000,0.333333,NaN,0.883688
3,헝가리와 관련된 올림픽 보이콧 사례는?,"[올림픽에서 첫 번째 보이콧은 1956년 하계 올림픽에서 시작되었다. 네덜란드, 스...","1956년 하계 올림픽에서 소련의 헝가리 침공에 항의하여 네덜란드, 스페인, 스위스...","1956년 하계 올림픽에서 네덜란드, 스페인, 스위스는 소련의 헝가리 침공에 항의해...",1.000000,1.000000,NaN,0.536039
4,아테네 올림픽에서 메달 말고 뭐 줬나요?,"[개인 혹은 팀으로 경기에 출전해서 1위, 2위, 3위를 한 선수는 메달을 받는다....",아테네(2004년) 올림픽에서는 메달과 함께 올리브 화환도 수여했습니다.,"아테네에서 열린 2004년 하계 올림픽 때는 1, 2, 3위 선수에게 메달과 함께 ...",NaN,1.000000,NaN,0.791445
5,1988년 하계 올림픽과 패럴림픽 개최 도시는 어떤 관계가 있나요?,[패럴림픽(Paralympic)은 신체·감각 장애가 있는운동 선수가 참가하는 국제 ...,1988년 서울 대회부터 하계 올림픽을 개최한 도시는 패럴림픽도 함께 개최하기로 하...,1988년 하계 올림픽부터 하계 올림픽을 개최한 도시는 패럴림픽도 같이 개최하기로 ...,1.000000,1.000000,NaN,0.725570
6,"패럴림픽이 1960년 하계 올림픽과 어떤 관련이 있으며, 이 대회가 패럴림픽 역사에...",[패럴림픽(Paralympic)은 신체·감각 장애가 있는운동 선수가 참가하는 국제 ...,정보가 포함된 문맥에 따르면 다음과 같습니다.\n\n- 관련성: 1960년 로마에서...,1960년 하계 올림픽이 열린 로마에서 구트만은 400명의 선수들을 'Paralle...,NaN,1.000000,NaN,0.531041
7,"1960년 하계 올림픽과 관련하여 패럴림픽이 어떻게 시작되었으며, 이 대회가 이후 ...",[패럴림픽(Paralympic)은 신체·감각 장애가 있는운동 선수가 참가하는 국제 ...,정보가 부족해 답을 할 수없습니다.,패럴림픽은 1948년 루드비히 구트만 경이 제2차 세계대전에 참전한 군인들의 사회 ...,1.000000,1.000000,0.0,0.000000
8,"1988년 하계 올림픽과 관련하여, 패럴림픽이 하계 올림픽과 같은 도시에서 개최되도...",[패럴림픽(Paralympic)은 신체·감각 장애가 있는운동 선수가 참가하는 국제 ...,문맥에 따르면 다음과 같습니다.\n\n- 역사적 배경: 패럴림픽의 기원은 1948년...,"패럴림픽은 신체 및 감각 장애가 있는 운동 선수가 참가하는 국제 스포츠 대회로, 1...",0.666667,1.000000,1.0,0.615915
9,"1904년 하계 올림픽에서 메달 수여 방식이 어떻게 바뀌었고, 그 이전 1896년 ...","[개인 혹은 팀으로 경기에 출전해서 1위, 2위, 3위를 한 선수는 메달을 받는다....",요약해서 답하면 다음과 같습니다.\n\n- 1896년 하계 올림픽: 메달이 2개만 ...,1904년 하계 올림픽부터 현재와 같은 메달 수여 방식이 시작되었어요. 이 방식에서...,0.800000,1.000000,1.0,0.000000
